# Construct Dataset

In this Jupyter notebook, the dataset going into the $mBasicPH\_storage$ is being constructed. Basically, this file constructs an excel file (stored in "EnergyEconGroupWork\Data") from the various excel files (stored in "EnergyEconGroupWork\DownloadDataForDK\ModelData") which were constructed from real world data.

## Settings

Import standard packages:

In [1]:
import pandas as pd,os, numpy as np

Let's specify an output folder:

In [2]:
output_dir = os.path.join(os.getcwd(),'Final_Dataset')

In [3]:
print(output_dir)

c:\Users\mpher\Documents\Uni\Master\02_Exchange\01_Academics\Energy Economics of the Green Transition\0_Final Project\EnergyEconGroupWork\DownloadDataForDK\ModelData\Final_Dataset


## Sheet "Log"

Here we construct the sheet "Log", which defines the **UNITS** of variables.

Define **UNITS**:

In [4]:
UNITS = {
    'FuelPrice':'EUR/MWh',
    'EmissionIntensity':'Ton CO2/MWh input',
    'EmissionTax':'EUR/TCO2',
    'Load':'MWh',
    'FuelMix':'MWh input / MWh output', # Data: 'TWh input / TWh output' -> no need to adjust ratio stays
    'GeneratingCapacity':'MW', 
    'OtherMC':'EUR/MWh output',
    'FOM':'EUR/(MW/(hours per model year))/8760', # convert from year to hours per model year
    'InvestCost':'Million EUR2015/MWhCapacity', # Data: 'Million EUR2015/GWhCapacity' -> adjust
    'LoadVariation':'Percent of annual demand',
    'CapVariation':'Percent of generating capacity',
    'MWP_E':'EUR/MWh',
    'MWP_H':'EUR/MWh',
    'E2H':'Coefficient (negative for heat pumps, positive for backpressure)'
}

Add dictionary to "Log" dataset:

In [5]:
df_Log = pd.DataFrame(list(UNITS.items()), columns=['Parameter', 'Unit/description'])

Save as excel:

In [6]:
df_Log.to_excel(os.path.join(output_dir,'Log.xlsx'),sheet_name='Log', index=False)

## Sheet "Fundamentals"

### FuelPrice

#### FuelPrice/BFt

Get different fuel types from "FuelMix" excel file in cwd.

In [7]:
BFt = pd.read_excel(os.path.join(os.getcwd(), 'FuelMix.xlsx'))

Subset:

In [8]:
BFt = BFt['BFt'].drop_duplicates()

Convert to df and set column name:

In [9]:
Fundamentals_df = pd.DataFrame({'FuelPrice/BFt': BFt})

In [10]:
Fundamentals_df

,FuelPrice/BFt
0,Biogas
2,Biomass
4,Coal
5,Natgas
7,Oil
10,Waste


#### FuelPrice/FuelPrice

We use the file "FuelProjections" (data from DEA) from the "EnergyEconomicsE2023" GitHub repository. Unfortunately, fuel prices are only stated from 2020 onwards. Therefore, we use the prices for 2020 as they are closest to 2019. We do so, because most fuel prices in the "FuelProjections" dataset increase over time, so we are the closest to 2019 prices by using the 2020 estimates.

In [11]:
FuelPrice = pd.read_excel(os.path.join(os.getcwd(), 'FuelProjections.xlsx'), sheet_name='prices')

In the FuelPrice df the prices are in EUR/GJ but we want EUR/MWh:

1 GJ = 0.2777777778 MWh ([Source](https://www.unitconverters.net/energy/gigajoule-to-megawatt-hour.htm))

Add to fundamentals df:

In [12]:
# Create empty new column in Fundamentals_df
Fundamentals_df['FuelPrice/FuelPrice'] = np.nan

In [13]:
# Biogas
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Biogas', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Biogas'] / 0.2777777778

# Biomass (we assume Biomass only consists of Wood pellets so we get close to prices in the mBasicPH_storageLarge)
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Biomass', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Wood pellets'] / 0.2777777778

# Coal
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Coal', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Coal'] / 0.2777777778
                                                                                                          
# Natgas
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Natgas', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Natural gas'] / 0.2777777778

# Oil
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Oil', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Oil'] / 0.2777777778

# Waste
Fundamentals_df.loc[Fundamentals_df['FuelPrice/BFt'] == 'Waste', 'FuelPrice/FuelPrice'] = FuelPrice.loc[0,'Waste'] / 0.2777777778


In [14]:
Fundamentals_df

,FuelPrice/BFt,FuelPrice/FuelPrice
0,Biogas,57.902680
2,Biomass,34.279615
4,Coal,7.339794
5,Natgas,13.265219
7,Oil,26.222730
10,Waste,0.048322


### EmissionIntensity

#### EmissionIntensity/BFt

Copy row "FuelPrice/BFt"

In [15]:
Fundamentals_df['EmissionIntensity/BFt'] = Fundamentals_df['FuelPrice/BFt']

#### EmissionIntensity/EmissionType

Fill new column with value "CO2", i.e. the same emission type for all fuels.

In [16]:
Fundamentals_df['EmissionIntensity/EmissionType'] = 'CO2'

#### EmissionIntensity/EmissionIntensity

As with the FuelPrice/FuelPrice, we use the file "FuelProjections" (data from DEA) from the "EnergyEconomicsE2023" GitHub repository. However, "EmissionIntensity" does not depend on time.

In [17]:
EmissionIntensity = pd.read_excel(os.path.join(os.getcwd(), 'FuelProjections.xlsx'), sheet_name='emissionIntensity')

In [18]:
EmissionIntensity

,EmissionIntensity/EmissionType/BFt,Coal,Oil,Natural gas,Straw,Wood pellets,Wood chips,Wood waste,Waste,Biogas,Hydrogen,Uranium
0,CO2,94.370,76.645000,57,0.0,0,0,0,42.500,0,0,0
1,SO2,0.272,0.159884,0,0.2,0,0,0,0.075,0,0,0


Add the emission intensity of CO2 to Fundamentals_df:

In the "EmissionIntensity" datafile above the values are in **kg/GJ** but we want **Ton CO2/MWh**. 

Thus, we devide by $1000*0.2777777778$. 

In [19]:
# Create empty new column in Fundamentals_df
Fundamentals_df['EmissionIntensity/EmissionIntensity'] = np.nan

# Biogas
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Biogas', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Biogas'] / (1000*0.2777777778)

# Biomass (we assume Biomass only consists of Wood pellets so we get close to prices in the mBasicPH_storageLarge)
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Biomass', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Wood pellets'] / (1000*0.2777777778)

# Coal
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Coal', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Coal'] / (1000*0.2777777778)
                                                                                                          
# Natgas
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Natgas', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Natural gas'] / (1000*0.2777777778)

# Oil
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Oil', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Oil'] / (1000*0.2777777778)

# Waste
Fundamentals_df.loc[Fundamentals_df['EmissionIntensity/BFt'] == 'Waste', 'EmissionIntensity/EmissionIntensity'] = EmissionIntensity.loc[0,'Waste'] / (1000*0.2777777778)

In [20]:
Fundamentals_df

,FuelPrice/BFt,FuelPrice/FuelPrice,EmissionIntensity/BFt,EmissionIntensity/EmissionType,EmissionIntensity/EmissionIntensity
0,Biogas,57.902680,Biogas,CO2,0.000000
2,Biomass,34.279615,Biomass,CO2,0.000000
4,Coal,7.339794,Coal,CO2,0.339732
5,Natgas,13.265219,Natgas,CO2,0.205200
7,Oil,26.222730,Oil,CO2,0.275922
10,Waste,0.048322,Waste,CO2,0.153000


### EmissionTax

We are goint to assume the average EU ETS price during the year 2019. The average EU Carbon Permits prices during 2019 was **24.64 EUR/TCO2** ([Source](https://tradingeconomics.com/commodity/carbon)).

We can add this information to the existing *Fundamentals_df*.

In [21]:
Fundamentals_df['EmissionTax/EmissionType'] = ['CO2'] + [np.nan] * (len(Fundamentals_df) - 1)
Fundamentals_df['EmissionTax/EmissionTax'] = [24.64] + [np.nan] * (len(Fundamentals_df) - 1)

### Save as excel

In [22]:
Fundamentals_df.to_excel(os.path.join(output_dir,'Fundamentals.xlsx'),sheet_name='Fundamentals', index=False)

## Sheet "LoadVariables"

*Note: Subtitles to previous section slightly differ (as we use less subtitles, i.e. subtitles per category and not per column as in "Sheet Fundamentals" section) to make the code more readable.*

### Electricity

We get load from "Load_E" file and only use rows "c_DK1" and "c_DK2" because we do not model transmission. In this step, we also already rename columns we want in our output df "LoadVariables_df".

In [23]:
Load_E = pd.read_excel(os.path.join(os.getcwd(), 'Load_E.xlsx')).rename(columns={
    'c_E':'Load_E/c_E',
    'Load_E':'Load_E/Load_E'})

Add the rows not conserning transmission to LoadVariables_df.

In [24]:
LoadVariables_df = Load_E[Load_E['Load_E/c_E'].isin(['c_DK1', 'c_DK2'])]

We get load variation from "LoadVariation_E" file. Again, we omit transmission.

In [25]:
LoadVariation_E = pd.read_excel(os.path.join(os.getcwd(), 'LoadVariation_E.xlsx')).rename(columns={
    'c_E':'LoadVariation_E/c_E',
    'h':'LoadVariation_E/h',
    'LoadVariation_E':'LoadVariation_E/LoadVariation_E'})

Add the rows not conserning transmission to LoadVariables_df.

In [26]:
LoadVariables_df = pd.concat([LoadVariables_df, LoadVariation_E[LoadVariation_E['LoadVariation_E/c_E'].isin(['c_DK1', 'c_DK2'])]], axis=1)

In [27]:
LoadVariables_df

,Load_E/c_E,Load_E/Load_E,LoadVariation_E/c_E,LoadVariation_E/h,LoadVariation_E/LoadVariation_E
0,c_DK1,2.037151e+07,c_DK1,1,0.000091
1,c_DK2,1.316852e+07,c_DK1,2,0.000089
2,NaN,NaN,c_DK1,3,0.000086
3,NaN,NaN,c_DK1,4,0.000084
4,NaN,NaN,c_DK1,5,0.000083
...,...,...,...,...,...
17515,NaN,NaN,c_DK2,8756,0.000131
17516,NaN,NaN,c_DK2,8757,0.000121
17517,NaN,NaN,c_DK2,8758,0.000115
17518,NaN,NaN,c_DK2,8759,0.000111


### Heat

We do the same steps as for electricity. But there is no export/import of heat.

In [28]:
Load_H = pd.read_excel(os.path.join(os.getcwd(), 'Load_H.xlsx')).rename(columns={
    'index':'Load_H/c_H',
    'Load_H':'Load_H/Load_H'})

In [29]:
LoadVariables_df = pd.concat([LoadVariables_df, Load_H], axis=1)

In [30]:
LoadVariation_H = pd.read_excel(os.path.join(os.getcwd(), 'LoadVariation_H.xlsx')).rename(columns={
    'c_H':'LoadVariation_H/c_H',
    'h':'LoadVariation_H/h',
    'LoadVariation_H':'LoadVariation_H/LoadVariation_H'})

In [31]:
LoadVariables_df = pd.concat([LoadVariables_df, LoadVariation_H], axis=1)

In [32]:
LoadVariables_df

,Load_E/c_E,Load_E/Load_E,LoadVariation_E/c_E,LoadVariation_E/h,LoadVariation_E/LoadVariation_E,Load_H/c_H,Load_H/Load_H,LoadVariation_H/c_H,LoadVariation_H/h,LoadVariation_H/LoadVariation_H
0,c_DK1,2.037151e+07,c_DK1,1.0,0.000091,c_DK1_Central,12759000.0,c_DK1_Central,1,0.000088
1,c_DK2,1.316852e+07,c_DK1,2.0,0.000089,c_DK1_LargeDecentral,3896000.0,c_DK1_Central,2,0.000079
2,NaN,NaN,c_DK1,3.0,0.000086,c_DK1_SmallDecentral,5906000.0,c_DK1_Central,3,0.000079
3,NaN,NaN,c_DK1,4.0,0.000084,c_DK2_Central,12200000.0,c_DK1_Central,4,0.000087
4,NaN,NaN,c_DK1,5.0,0.000083,c_DK2_LargeDecentral,2017000.0,c_DK1_Central,5,0.000117
...,...,...,...,...,...,...,...,...,...,...
52555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_DK2_SmallDecentral,8756,0.000165
52556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_DK2_SmallDecentral,8757,0.000165
52557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_DK2_SmallDecentral,8758,0.000150
52558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c_DK2_SmallDecentral,8759,0.000120


### Save as excel

In [33]:
LoadVariables_df.to_excel(os.path.join(output_dir,'LoadVariables.xlsx'),sheet_name='LoadVariables', index=False)

## Sheet "LoadMaps"

We take "c_E" columns from before created "LoadVariables" excel file.

In [34]:
LoadMaps_df = pd.read_excel(os.path.join(output_dir, 'LoadVariables.xlsx'),
                            usecols=['Load_E/c_E', 'Load_H/c_H']).dropna(subset=['Load_H/c_H']).rename(columns={
                                'Load_E/c_E':'c_E2g_E/c_E',
                                'Load_H/c_H':'c_H2g_H/c_H'})

Define a function to check if "DK1" or "DK2" is included in the just created columns


In [35]:
def check_dk(value):
    if isinstance(value, str):
        if 'DK1' in value:
            return 'DK1'
        elif 'DK2' in value:
            return 'DK2'
    return np.nan

Apply function to creat new columns "c_E2g/g" and "c_H2g/g":

In [36]:
LoadMaps_df['c_E2g_E/g_E'] = LoadMaps_df['c_E2g_E/c_E'].apply(check_dk)
LoadMaps_df['c_H2g_H/g_H'] = LoadMaps_df['c_H2g_H/c_H'].apply(check_dk)

Rearrange columns:

In [37]:
LoadMaps_df = LoadMaps_df[['c_E2g_E/c_E','c_E2g_E/g_E','c_H2g_H/c_H','c_H2g_H/g_H']]

### Save as excel

In [38]:
LoadMaps_df.to_excel(os.path.join(output_dir,'LoadMaps.xlsx'),sheet_name='LoadMaps', index=False)

## Sheet "MarketMaps"

We take the market maps from *E42_Data* and store it in our folder *Final_Dataset*.

## Sheet "GeneratorsVariables"

### FuelMix

Import from "FuelMix.xlsx".

In [39]:
FuelMix = pd.read_excel(os.path.join(os.getcwd(), 'FuelMix.xlsx')).rename(columns={
    'id':'FuelMix/id',
    'BFt':'FuelMix/BFt',
    'FuelMix':'FuelMix/FuelMix'})

Remove "id_" prefix from "FuelMix/id" column

In [40]:
FuelMix['FuelMix/id'] = FuelMix['FuelMix/id'].str.replace('id_', '')

### GeneratingCap Electricity

In [41]:
GeneratingCap_E = pd.read_excel(os.path.join(os.getcwd(), 'GeneratingCapacity_E.xlsx')).rename(columns={
    'id':'GeneratingCap_E/id',
    'GeneratingCapacity_E':'GeneratingCap_E/GeneratingCap_E'})

In [42]:
GeneratingCap_E['GeneratingCap_E/id'] = GeneratingCap_E['GeneratingCap_E/id'].str.replace('id_', '')

In [43]:
# Filter out rows containing "ImportFrom"
GeneratingCap_E = GeneratingCap_E[~GeneratingCap_E['GeneratingCap_E/id'].str.contains('ImportFrom')]

### GeneratingCap Heat

In [44]:
GeneratingCap_H = pd.read_excel(os.path.join(os.getcwd(), 'GeneratingCapacity_H.xlsx')).rename(columns={
    'id':'GeneratingCap_H/id',
    'GeneratingCapacity_H':'GeneratingCap_H/GeneratingCap_H'})

In [45]:
GeneratingCap_H['GeneratingCap_H/id'] = GeneratingCap_H['GeneratingCap_H/id'].str.replace('id_', '')

### OtherMC

Importing while only keeping first row. We only keep the first row, because expect for importing MC, which we are disregarding, MC are the same across all hours.

In [46]:
OtherMC = pd.read_excel(os.path.join(os.getcwd(), 'OtherMC.xlsx')).head(1).drop(columns='h')

Filter out importing MC:

In [47]:
OtherMC = OtherMC.filter(regex='^(?!.*ImportFrom).*$', axis=1)

Transfer df from wide to long:

In [48]:
OtherMC = OtherMC.melt(var_name='OtherMC/id', value_name='OtherMC/OtherMC')

Drop "id" again:

In [49]:
OtherMC['OtherMC/id'] = OtherMC['OtherMC/id'].str.replace('id_', '')

### FOM

In [50]:
FOM = pd.read_excel(os.path.join(os.getcwd(), 'FOM.xlsx')).rename(columns={
    'id':'FOM/id',
    'FOM':'FOM/FOM'})

In [51]:
FOM['FOM/id'] = FOM['FOM/id'].str.replace('id_', '')

Filter out importing MC:

In [52]:
# Filter out rows containing "ImportFrom"
FOM = FOM[~FOM['FOM/id'].str.contains('ImportFrom')]

### InvestCost

We abstract from investment costs in generators.

### E2H

In [53]:
E2H = pd.read_excel(os.path.join(os.getcwd(), 'E2H.xlsx')).rename(columns={
    'id':'E2H/id',
    'E2H':'E2H/E2H'})

In [54]:
E2H['E2H/id'] = E2H['E2H/id'].str.replace('id_', '')

### Put dataframes together

In [55]:
GeneratorsVariables_df = pd.concat([FuelMix,GeneratingCap_E,GeneratingCap_H,OtherMC,FOM,E2H], axis=1)

### Save as excel

In [56]:
GeneratorsVariables_df.to_excel(os.path.join(output_dir,'GeneratorsVariables.xlsx'),sheet_name='GeneratorsVariables', index=False)

## Sheet "GeneratorsMaps"

## Combine excel files

In this last step, we combine the different excel files into one excel file split up into multiple sheets.

In [57]:
# List to store DataFrames from each file
#df_final = []

# Iterate over each file in the directory
#for file in os.listdir(output_dir):
#    if file.endswith('.xlsx'):
#        file_path = os.path.join(output_dir, file)
        # Read each Excel file into a DataFrame
#        df = pd.read_excel(file_path)
        # Append DataFrame to the list
#        df_final.append(df)

# Create a new Excel file with multiple sheets
#with pd.ExcelWriter(output_dir, engine='xlsxwriter') as writer:
    # Iterate over DataFrames and write each to a separate sheet
#    for i, df in enumerate(df_final):
#        sheet_name = f'Sheet_{i+1}'
#        df.to_excel(writer, sheet_name=sheet_name, index=False)